#### Import Packages

In [123]:
# dataset retrieved from https://archive.ics.uci.edu/dataset/352/online+retail
import pandas as pd
import numpy as np
import plotly.express as px

#### Import Data

In [124]:
file_path = "data/Online_Retail.csv"


df = pd.read_csv(file_path,
                 header= 0,
                #  nrows= 5,
                 parse_dates= ["InvoiceDate"],
                 date_format= "%d/%m/%y %H:%M", # account for DD/MM/YY HH:MM
                 converters= {'UnitPrice': lambda pounds: int(float(pounds) * 100)}
                 )

# print(df.info())
# print(df.head())

#### Account for null values
- 1,454 values in description (dropped)
- 135,080 values in CustomerID, likely guests

540,455 rows left after dropping NA values in description

In [125]:
print("Number of NA values in each column:")
print(df.isna().sum())

total_na = df.isna().sum().sum()
print(f"Total number of NA values in the DataFrame: {total_na}")

# drop rows without descriptions
df = df.dropna(subset=['Description'])

print(df.isna().sum())
# print(len(df))

Number of NA values in each column:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
Total number of NA values in the DataFrame: 136534
InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133626
Country             0
dtype: int64


#### Split Dataset
- valid_sales (531167 rows)
- cancellations (9288 rows)


In [126]:
# sort valid sales from cancellations based on the starting 'C'
valid_sales = df[~df['InvoiceNo'].str.startswith('C')] # ~ is the bitwise NOT operator
cancellations = df[df['InvoiceNo'].str.startswith('C')]

# print(len(valid_sales))
# print(len(cancellations))

#### Valid Sales

In [127]:
# print(cancellations[cancellations['StockCode'].str.len() < 5])
# print(valid_sales[valid_sales['StockCode'].str.len() < 5])

#### Cleaning valid_sales
- Items with quantities of 0 or less removed / 474 removed
- Items with StockCode describing postage e.g. POST, M etc. removed / 2,312 removed
- Items with invalid price (£0 or less) removed / 578 removed

527,803 rows left

In [128]:
# print(valid_sales.info())
print(len(valid_sales), "rows before cleaning") # 531,167 rows initially
valid_sales = valid_sales[valid_sales['Quantity'] >= 1] # keep valid quantities | 474 rows removed
# print(len(valid_sales)) # 530,693 rows left
valid_sales = valid_sales[valid_sales['StockCode'].str.len() >= 5] # keep stockcodes for valid items | 2312 rows removed
# print(len(valid_sales)) # 528,381 rows left
valid_sales = valid_sales[valid_sales['UnitPrice'] > 0] # remove any items with price £0 or less | 578 rows removed
# print(len(valid_sales)) # 527,803 rows left

print(len(valid_sales), "rows after cleaning") # 527,803 rows left

531167 rows before cleaning
527802 rows after cleaning


#### Cleaning cancellations

In [129]:
# print(cancellations.info())
# print(cancellations.head())
print(len(cancellations), "rows before cleaning") # 9,288 rows initially
cancellations['Quantity'] = cancellations['Quantity'].abs() # convert all quantities to positive
print(len(cancellations)) # 9,288 rows left
cancellations = cancellations[cancellations['StockCode'].str.len() >= 5] # keep stockcodes for valid items | 527 rows removed
print(len(cancellations)) # 8,761 rows left
cancellations = cancellations[cancellations['UnitPrice'] > 0] # remove any items with price £0 or less | 0 rows removed
print(len(cancellations)) # 8,761 rows left

9288 rows before cleaning
9288
8761
8761


/var/folders/xz/hvt4xjsx52d4vvbdqkszth7h0000gn/T/ipykernel_9835/3195878376.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



##### General Analysis

In [130]:
total_valid_sales = len(valid_sales)
total_cancellations = len(cancellations)

print(total_valid_sales) # 527,803 rows
print(total_cancellations) # 8,761 rows

fig_proportion_cancelled = px.pie(values= [total_valid_sales, total_cancellations],
                                  names= ['Valid Sales', 'Cancellations'],
                                  title= 'Proportion of Cancelled Orders')

fig_proportion_cancelled.update_layout(
    title={
        'text': 'Proportion of Cancelled Orders',
        'x': 0.5,                   # x position of title (0 is left, 1 is right)
        'xanchor': 'center'         # Center of title aligned with x=0.5
    }
)

fig_proportion_cancelled.show()

527802
8761


##### Sales Analysis
Revenue Analysis (By Month)

In [136]:
# New column to reflect revenue earned from each sale
valid_sales["Revenue"] = valid_sales['Quantity'] * valid_sales['UnitPrice']

print("Total revenue (GBP):", valid_sales['Revenue'].sum() / 100)

# New column YearMonth to separate sales by monthly buckets
valid_sales['YearMonth'] = valid_sales['InvoiceDate'].dt.to_period('M') ## YearMonth in format YYYY-MM

# print(valid_sales.head())
#* Sum up revenue columns based on buckets of YearMonth
monthly_sales = valid_sales.groupby('YearMonth')['Revenue'].sum().reset_index()
monthly_sales['YearMonth'] = monthly_sales['YearMonth'].astype(str) # convert to str for plotly

# print(monthly_sales)
#* identify best/worst performing month
highest_revenue_row = monthly_sales.loc[monthly_sales['Revenue'].idxmax()]
lowest_revenue_row  = monthly_sales.loc[monthly_sales['Revenue'].idxmin()]


print(highest_revenue_row)
print(lowest_revenue_row)


print("Average revenue (GBP):", monthly_sales['Revenue'].mean() / 100)

fig_monthly_sales = px.line(monthly_sales,
              x = 'YearMonth',
              y = monthly_sales['Revenue'] / 100,
              title = "Total Revenue by Month",
              labels = {'YearMonth': 'Date', 'y':'Revenue (Pounds Sterling)'},
              markers= True)

# Center the title
fig_monthly_sales.update_layout(
    title={
        'text': 'Total Revenue by Month',
        'x': 0.5,                   # x position of title (0 is left, 1 is right)
        'xanchor': 'center'         # Center of title aligned with x=0.5
    }
)

fig_monthly_sales.show()

Total revenue (GBP): 10282661.99
YearMonth      2011-11
Revenue      145729707
Name: 11, dtype: object
YearMonth     2011-02
Revenue      50877824
Name: 2, dtype: object
Average revenue (GBP): 790973.9992307692


Revenue Analysis (By Country)

In [137]:

# print(valid_sales['Country'].unique())
# print(valid_sales[valid_sales['Country'] == "Unspecified"]) # 446 items aren't associated with any country
# * group by 'Country', perform aggregation on Revenue, Quantity, InvoiceNo
country_analysis = valid_sales.groupby('Country').agg(
    TotalRevenue=('Revenue', 'sum'),
    TotalQuantity=('Quantity', 'sum'),
    TotalInvoices=('InvoiceNo', 'nunique')
).reset_index()

# Calculate Average Order Value (AOV) in GBP i.e. average revenue per transaction / invoice
country_analysis['AOV'] = country_analysis['TotalRevenue'] / (country_analysis['TotalInvoices'] * 100)

# Sort countries by revenue (desc)
country_analysis = country_analysis.sort_values(by='TotalRevenue', ascending=False)

# Top 10 countries by revenue
top_ten_revenue = country_analysis.head(10)

print(top_ten_revenue)

fig_top_ten_revenue = px.bar(top_ten_revenue,
                     x = 'Country',
                     y = top_ten_revenue['TotalRevenue'] / 100,
                     labels = {'Country': 'Country', 'y':'Revenue (Pounds Sterling)'},
                     color = 'Country',
                     title= 'Top 10 Total Revenue (by Country)',
                     text_auto= True)

fig_top_ten_revenue.update_layout(
    title={
        'text': 'Top 10 Total Revenue (by Country)',
        'x': 0.5,                   # x position of title (0 is left, 1 is right)
        'xanchor': 'center'         # Center of title aligned with x=0.5
    }
)

fig_top_ten_revenue.show()

           Country  TotalRevenue  TotalQuantity  TotalInvoices          AOV
36  United Kingdom     875934153        4654401          17914   488.966257
24     Netherlands      28379469         200258             93  3051.555806
10            EIRE      27102833         147062            282   961.093369
14         Germany      20551567         118139            443   463.917991
13          France      18452337         111269            382   483.045471
0        Australia      13813770          83900             56  2466.744643
31           Spain       5570947          27731             88   633.062159
33     Switzerland       5307135          30527             50  1061.427000
20           Japan       3738506          26016             19  1967.634737
3          Belgium       3691699          22962             98   376.703980


Top Average Order Values (Top 10)

In [135]:
country_analysis = country_analysis.sort_values(by='AOV', ascending=False)

# Top 10 countries by AOV
top_ten_AOV = country_analysis.head(10)

# print(top_ten_AOV)

fig_top_ten_aov = px.bar(top_ten_AOV,
                     x = 'Country', y = 'AOV',
                     labels = {'Country': 'Country', 'AOV':'Average Order Value (Pound Sterling)'},
                     color = 'Country',
                     title= 'Top 10 Average Order Values (by Country)',
                     text_auto= True)

fig_top_ten_aov.update_layout(
    title={
        'text': 'Top 10 Average Order Values (by Country)',
        'x': 0.5,                   # x position of title (0 is left, 1 is right)
        'xanchor': 'center'         # Center of title aligned with x=0.5
    }
)

fig_top_ten_aov.show()